#  causality sentence Identifier

## 1.  load sentence

In [1]:
###semeval###

#load semeval causalityDetection data


import pandas as pd
#sem_CE_test= pd.read_csv('data/in/semeval/semeval_causalityDetection.csv')
sem_CE_test = pd.read_csv('data/in/KNN_RAG/semeval_causalityDetection_cleaned.csv')

pm_DB = sem_CE_test
pm_DB['label'] = pm_DB['label'].astype(int)
pm_DB

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings
0,0,0,0,Dogs develop a fever from stress and/or pain s...,1,"[-0.0036133944522589445, 0.0012565960641950369..."
1,1,1,1,The new pencil and graphite drawing is the pai...,0,"[-0.02040475234389305, 0.022646330296993256, 0..."
2,2,2,2,The deleveraging triggered by the crisis impli...,1,"[-0.02002304047346115, -0.031225474551320076, ..."
3,3,3,3,The fat and cholesterol cause heart disease; t...,1,"[0.002320877043530345, -0.014957467094063759, ..."
4,4,4,4,"The researchers placed the compound in a tube,...",0,"[-0.019478240981698036, 0.0026719982270151377,..."
...,...,...,...,...,...,...
544,544,621,621,Neuralgia (TN) is the most common cause of chr...,1,"[-0.010079591535031796, 0.014008276164531708, ..."
545,545,622,622,Muscle fatigue is the number one cause of arm ...,1,"[-0.011783338151872158, -0.007658560294657946,..."
546,546,623,623,The movie featured a mutated sheep as the titu...,0,"[-0.015397767536342144, -0.01711292937397957, ..."
547,547,624,624,A plasma was generated by an electric discharg...,1,"[-0.011884856037795544, 0.02052135579288006, 0..."


In [2]:
###few shot example retriever.
# 1. if pattern matched give matched examples as few shot. else give random 10 examples

input_sentence = pm_DB['sentence'][0]
input_sentence

'Dogs develop a fever from stress and/or pain such as in a severe flea infestation.'

## 2. load fewshot database

In [3]:
#load semeval sentence and patterns.
semeval_data = pd.read_csv('data/in/RAG/comb_SCITEsemADE_CausalityPattern.csv')# 

semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

## 3. get causality label for sentence

In [4]:
from openai import OpenAI
client = OpenAI(api_key='enter your API key here..')


In [5]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        window_size = len(p)

        # Slide through input_sentence with a window size of 8
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('p is ',p, 'i is', i)
                #print(f"Window '{window}' - Levenshtein Score: {score}")
    fewshot_index = list(set(fewshot_index))
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        #fewshot_index= fewshot_index[:10]
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [6]:

def get_fewshotexamples(fewshot_index):


    train_sentence = semeval_data['sentence'].to_list()

    fewshot_examples = [train_sentence[i] for i in fewshot_index]

    import json
    fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_examples



In [7]:
def get_randomfewshotexamples():
    train_sentence = semeval_data['sentence'].to_list()

    rand_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120]
    #rand_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120, 3, 9, 6, 140, 130, 160, 250, 350, 500, 900]

    #fewshot_examples = [train_sentence[i] for i in range(10)]
    fewshot_examples = [train_sentence[i] for i in rand_index]

    import json
    fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_examples
    
    

In [8]:
fewshot_index = get_fewshot_examples_index(input_sentence)
fewshot_examples = get_fewshotexamples(fewshot_index)    

In [9]:
#Below are 10 example sentences which shows causality relation  with  their cause and effect 
          #phrase delimited between the XML tags <cause>, </cause> , <effect> and </effect>.
           #+ fewshot_examples
###model#gpt-3.5-turbo
def CS_Identifier(input_sentence, fewshot_examples):
    
    response = client.chat.completions.create(
      model="gpt-4o",
      response_format= {"type": "json_object"},
      messages=[
        {
          "role": "system",
          "content": '''
          You will be given a sentence, and your task is to identify whether the sentence shows a 
          causality relation or not. If the sentence shows a causality relation, you can output 1;
          if it doesn't, output 0. 
          Your output should be in json format: {output :  }.
          
          Hint: If the sentence shows causality relation, you will be able to identify a pair of 
          phrase in the sentence where one phrase will be the cause of the other phrase(effect).
          
          Below are 10 example sentences which shows causality relation  with  their cause and effect 
          phrase delimited between the XML tags <cause>, </cause> , <effect> and </effect>.
           '''+ fewshot_examples
          
            
        },
        {
          "role": "user",
          "content": input_sentence 
        }
      ],
      temperature=0.0,
      seed = 1,
      max_tokens=100,
      top_p=1
    )
    
    return response.choices[0].message.content


In [15]:
import json
causality_identifier=[]
fewshotfound_inputsentence_index=[]
fewshot_examples_list=[]
for i in range (len(pm_DB)):
    input_sentence = pm_DB['sentence'][i]
    # few shot examples augmentation.
    #fewshot_index = get_fewshot_examples_index(input_sentence)############FEWSHOT
    if len(fewshot_index) == 0:
        fewshot_examples =  get_randomfewshotexamples()
    else:
        fewshotfound_inputsentence_index.append(i)
        #fewshot_examples = get_fewshotexamples(fewshot_index) #############fewshot
        fewshot_examples =  get_randomfewshotexamples()##################random shot/zero shot
        
    #print('input_sentence : ', input_sentence)

    print('fewshot_examples : ', fewshot_examples)
    fewshot_examples_list.append(fewshot_examples)
    # output generation
    out = CS_Identifier(input_sentence, fewshot_examples)
    json_out = json.loads(out)
    causality_identifier_out = json_out['output']
    causality_identifier.append(int(causality_identifier_out))
    
    
    print(i)
    print(causality_identifier_out)
    
    

fewshot_examples :  {"example 1": "<cause>Flecainide</cause>-associated pneumonitis with <effect>acute respiratory failure</effect> in a patient with the LEOPARD syndrome.", "example 2": "<cause>Propecia</cause>-associated <effect>bilateral cataract</effect>.", "example 3": "<cause>Lansoprazole</cause>-associated <effect>collagenous colitis</effect>: a case report.", "example 4": "Intravenous <cause>azithromycin</cause>-induced <effect>ototoxicity</effect>.", "example 5": "A 60-year-old woman with diabetes mellitus (type 2) developed an <effect>acute icteric hepatitis-like illness</effect> 6 weeks after the initiation of <cause>gliclazide</cause> therapy.", "example 6": "Sodium phenylbutyrate was added to treatment with <cause>HU</cause> in two patients, but failed to produce an increase in total Hb despite <effect>increasing Hb F</effect> levels.", "example 7": "<effect>Cutaneous eruptions</effect> occur in 3% of individuals administered <cause>carbamazepine</cause>.", "example 8": "T

2
1
fewshot_examples :  {"example 1": "<cause>Flecainide</cause>-associated pneumonitis with <effect>acute respiratory failure</effect> in a patient with the LEOPARD syndrome.", "example 2": "<cause>Propecia</cause>-associated <effect>bilateral cataract</effect>.", "example 3": "<cause>Lansoprazole</cause>-associated <effect>collagenous colitis</effect>: a case report.", "example 4": "Intravenous <cause>azithromycin</cause>-induced <effect>ototoxicity</effect>.", "example 5": "A 60-year-old woman with diabetes mellitus (type 2) developed an <effect>acute icteric hepatitis-like illness</effect> 6 weeks after the initiation of <cause>gliclazide</cause> therapy.", "example 6": "Sodium phenylbutyrate was added to treatment with <cause>HU</cause> in two patients, but failed to produce an increase in total Hb despite <effect>increasing Hb F</effect> levels.", "example 7": "<effect>Cutaneous eruptions</effect> occur in 3% of individuals administered <cause>carbamazepine</cause>.", "example 8"

5
0
fewshot_examples :  {"example 1": "<cause>Flecainide</cause>-associated pneumonitis with <effect>acute respiratory failure</effect> in a patient with the LEOPARD syndrome.", "example 2": "<cause>Propecia</cause>-associated <effect>bilateral cataract</effect>.", "example 3": "<cause>Lansoprazole</cause>-associated <effect>collagenous colitis</effect>: a case report.", "example 4": "Intravenous <cause>azithromycin</cause>-induced <effect>ototoxicity</effect>.", "example 5": "A 60-year-old woman with diabetes mellitus (type 2) developed an <effect>acute icteric hepatitis-like illness</effect> 6 weeks after the initiation of <cause>gliclazide</cause> therapy.", "example 6": "Sodium phenylbutyrate was added to treatment with <cause>HU</cause> in two patients, but failed to produce an increase in total Hb despite <effect>increasing Hb F</effect> levels.", "example 7": "<effect>Cutaneous eruptions</effect> occur in 3% of individuals administered <cause>carbamazepine</cause>.", "example 8"

KeyboardInterrupt: 

In [11]:
1524

1524

In [12]:
len(causality_identifier)

549

In [13]:
pm_DB['GPT_causality_label'] = causality_identifier
pm_DB['fewshot_examples'] = fewshot_examples_list

#pm_DB['patternmatcher_label'] = pattern_matched 
pm_DB

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label,fewshot_examples
0,0,0,0,Dogs develop a fever from stress and/or pain s...,1,"[-0.0036133944522589445, 0.0012565960641950369...",1,"{""example 1"": ""<cause>Flecainide</cause>-assoc..."
1,1,1,1,The new pencil and graphite drawing is the pai...,0,"[-0.02040475234389305, 0.022646330296993256, 0...",0,"{""example 1"": ""<cause>Flecainide</cause>-assoc..."
2,2,2,2,The deleveraging triggered by the crisis impli...,1,"[-0.02002304047346115, -0.031225474551320076, ...",1,"{""example 1"": ""<cause>Flecainide</cause>-assoc..."
3,3,3,3,The fat and cholesterol cause heart disease; t...,1,"[0.002320877043530345, -0.014957467094063759, ...",1,"{""example 1"": ""<cause>Flecainide</cause>-assoc..."
4,4,4,4,"The researchers placed the compound in a tube,...",0,"[-0.019478240981698036, 0.0026719982270151377,...",0,"{""example 1"": ""<cause>Flecainide</cause>-assoc..."
...,...,...,...,...,...,...,...,...
544,544,621,621,Neuralgia (TN) is the most common cause of chr...,1,"[-0.010079591535031796, 0.014008276164531708, ...",1,"{""example 1"": ""<cause>Flecainide</cause>-assoc..."
545,545,622,622,Muscle fatigue is the number one cause of arm ...,1,"[-0.011783338151872158, -0.007658560294657946,...",1,"{""example 1"": ""<cause>Flecainide</cause>-assoc..."
546,546,623,623,The movie featured a mutated sheep as the titu...,0,"[-0.015397767536342144, -0.01711292937397957, ...",0,"{""example 1"": ""<cause>Flecainide</cause>-assoc..."
547,547,624,624,A plasma was generated by an electric discharg...,1,"[-0.011884856037795544, 0.02052135579288006, 0...",0,"{""example 1"": ""<cause>Flecainide</cause>-assoc..."


In [14]:
#get accuracy for fewshot_semeval_causalityDetection: b/w label and GPT_causality_label 

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

# Calculate precision, recall, and F1 score
precision = precision_score(pm_DB['label'], pm_DB['GPT_causality_label'])
recall = recall_score(pm_DB['label'], pm_DB['GPT_causality_label'])
f1 = f1_score(pm_DB['label'], pm_DB['GPT_causality_label'])
accuracy = accuracy_score(pm_DB['label'], pm_DB['GPT_causality_label'])

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.8633879781420765
F1 Score: 0.8366013071895425
Precision: 0.9142857142857143
Recall: 0.7710843373493976


In [ ]:

#filtered_rows = pm_DB[(pm_DB['GPT_causality_label'] == 1) & (pm_DB['patternmatcher_label'] == 1)]
#filtered_rows.to_csv('data/out/alzeimers_causalityrelation.csv')

#pm_DB.to_csv('data/out/RAG_causality/semeval/fewshotcomb_semeval_causalityDetection.csv')

In [ ]:
#pm_DB.to_csv('data/out/RAG_causality/task1/GPT4/zero_SCITE_causalityDetection.csv')#RAG, rand,zero, ADE, SCITE, sem

In [ ]:
pm_DB.to_csv('data/out/RAG_causality/task1/errorAnalysis/RAG_sem_causalityDetection.csv')#RAG, rand,zero, ADE, SCITE, sem